In [1]:
#필요 패키지 설치
!pip install numpy==1.23.5
!pip install gluonnlp pandas tqdm
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch
#KoBERT 깃허브에서 불러오기
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Using cached gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl
  Using cached mxnet-1.9.1-py3-none-manylinux2014_x86_64.whl (49.1 MB)
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 6.2 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-wshr1f1w
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-wshr1f1w
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 12.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of kobert to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement onnxruntime<=1.8.0,==1.8.0 (from kobert) (from versions: 1.12.0, 1.12.1, 1.13.1, 1.14.0, 1.14.1, 1.15.0, 1.15.1, 1.16.0, 1.16.1, 1.16.2, 1.16.3, 1.17.0, 1.17.1, 1.17.3, 1.18.0)
ERROR: No matching distribution found for onnxruntime<=1.8.0,==1.8.0


In [2]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")
bertmodel = AutoModel.from_pretrained("skt/kobert-base-v1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from transformers import BertModel, BertTokenizer, AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

# Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer

device = torch.device("cuda:0")

In [4]:
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

In [44]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/5차년도_2차.csv', encoding='cp949')
data = data.loc[:, ['wav_id', '발화문', '상황']]
data.head()

,wav_id,발화문,상황
0,5f4141e29dd513131eacee2f,헐! 나 이벤트에 당첨 됐어.,happiness
1,5f4141f59dd513131eacee30,내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.,happiness
2,5f4142119dd513131eacee31,"한 명 뽑는 거였는데, 그게 바로 내가 된 거야.",happiness
3,5f4142279dd513131eacee32,"당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. ...",happiness
4,5f3c9ed98a3c1005aa97c4bd,에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.,neutral


In [45]:
#데이터를 긍정/부정/중립으로 구분
#부정: 공포, 분노, 슬픔, 혐오 => 0
#긍정: 놀람, 행복 => 1
#중립: 중립 => 2

data.loc[(data['상황'] == "fear"), '상황'] = 0  #공포 => 0
data.loc[(data['상황'] == "surprise"), '상황'] = 1  #놀람 => 1
data.loc[(data['상황'] == "angry"), '상황'] = 0  #분노 => 0
data.loc[(data['상황'] == "sadness"), '상황'] = 0  #슬픔 => 0
data.loc[(data['상황'] == "neutral"), '상황'] = 2  #중립 => 2
data.loc[(data['상황'] == "happiness"), '상황'] = 1  #행복 => 1
data.loc[(data['상황'] == "disgust"), '상황'] = 0  #혐오 => 0

In [46]:
data_list = []
for ques, label in zip(data['발화문'], data['상황'])  :
    data = []
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

In [47]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))


    def __len__(self):
        return (len(self.labels))

In [48]:
# Setting parameters
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 2e-5

In [49]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)
dataset_train, dataset_val = train_test_split(dataset_train, test_size=0.2, shuffle=True, random_state=34)

In [53]:
tok = tokenizer.tokenize

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_val = BERTDataset(dataset_val, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [54]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
val_dataloader = torch.utils.data.DataLoader(data_val, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [55]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [56]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [57]:
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [58]:
#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [59]:
train_history=[]
val_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    val_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(val_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        val_acc += calc_accuracy(out, label)
    print("epoch {} val acc {}".format(e+1, val_acc / (batch_id+1)))
    val_history.append(val_acc / (batch_id+1))

<ipython-input-59-a151a4af694a>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/388 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 1 batch id 1 loss 1.1713343858718872 train acc 0.1875
epoch 1 batch id 201 loss 0.447385311126709 train acc 0.6924751243781094
epoch 1 train acc 0.7319856099656358


<ipython-input-59-a151a4af694a>:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(val_dataloader)):


...

epoch 4 val acc 0.8193114874815907


  0%|          | 0/388 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.2578342854976654 train acc 0.84375
epoch 5 batch id 201 loss 0.3386146128177643 train acc 0.8728233830845771
epoch 5 train acc 0.8732764175257732


  0%|          | 0/97 [00:00<?, ?it/s]

epoch 5 val acc 0.8232234904270986


In [60]:
torch.save(model, '/content/drive/MyDrive/Colab Notebooks/tuning3/kobert3_tuning.pt')

In [61]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/tuning3/kobert3_tuning_state_dict.pt')

In [62]:
torch.save({
    'model':model.state_dict(),
    'optimizer':optimizer.state_dict()
}, '/content/drive/MyDrive/Colab Notebooks/tuning3/kobert3_tuning_all.tar')

In [63]:
from sklearn.metrics import classification_report

In [64]:
# 예측 함수 정의
def predict(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch_id, (token_ids, valid_length, segment_ids, labels) in enumerate(dataloader):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            labels = labels.long().to(device)

            outputs = model(token_ids, valid_length, segment_ids)
            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_preds, all_labels

# 클래스별 성능 평가 함수 정의
def evaluate_classwise(model, dataloader, device):
    preds, labels = predict(model, dataloader, device)
    report = classification_report(labels, preds, target_names=['부정', '긍정', '무감정'])
    print(report)

In [66]:
evaluate_classwise(model, test_dataloader, device)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


              precision    recall  f1-score   support

          부정       0.81      0.86      0.84      1946
          긍정       0.81      0.80      0.81      1267
         무감정       0.75      0.62      0.68       662

    accuracy                           0.80      3875
   macro avg       0.79      0.76      0.77      3875
weighted avg       0.80      0.80      0.80      3875



In [67]:
# 클래스별 성능 평가 함수 정의
def evaluate_classwise(model, dataloader, device, class_names):
    preds, labels = predict(model, dataloader, device)
    report = classification_report(labels, preds, target_names=class_names, output_dict=True)
    df_report = pd.DataFrame(report).transpose()

    print("Class-wise Performance Metrics:")
    print(df_report)

    accuracies = df_report.loc[class_names, 'precision'].to_dict()
    for cls in class_names:
        print(f"Accuracy for class {cls}: {accuracies[cls]*100:.2f}%")

    return labels, preds

In [68]:
# 잘못 예측한 경우 요약 함수 정의
def misclassified_cases_summary(labels, preds, class_names):
    data = {'True Label': labels, 'Predicted Label': preds}
    df = pd.DataFrame(data)
    misclassified = df[df['True Label'] != df['Predicted Label']]
    misclassified_summary = misclassified.groupby(['True Label', 'Predicted Label']).size().reset_index(name='Count')
    misclassified_summary = misclassified_summary.sort_values(by='Count', ascending=False).reset_index(drop=True)

    misclassified_summary['True Label'] = misclassified_summary['True Label'].apply(lambda x: class_names[x])
    misclassified_summary['Predicted Label'] = misclassified_summary['Predicted Label'].apply(lambda x: class_names[x])

    print("\nMisclassified Cases Summary:")
    print(misclassified_summary)

In [69]:
# 사용할 클래스 이름들
class_names = ['부정', '긍정', '무감정']

In [70]:
labels, preds = evaluate_classwise(model, test_dataloader, device, class_names)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Class-wise Performance Metrics:
              precision    recall  f1-score      support
부정             0.811805  0.862282  0.836282  1946.000000
긍정             0.809221  0.803473  0.806337  1267.000000
무감정            0.749091  0.622356  0.679868   662.000000
accuracy       0.802065  0.802065  0.802065     0.802065
macro avg      0.790039  0.762704  0.774162  3875.000000
weighted avg   0.800246  0.802065  0.799769  3875.000000
Accuracy for class 부정: 81.18%
Accuracy for class 긍정: 80.92%
Accuracy for class 무감정: 74.91%


In [72]:
misclassified_cases_summary(labels, preds, class_names)


Misclassified Cases Summary:
  True Label Predicted Label  Count
0         긍정              부정    201
1        무감정              부정    188
2         부정              긍정    178
3         부정             무감정     90
4        무감정              긍정     62
5         긍정             무감정     48


In [21]:
def predict_sentence(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
              test_eval.append('부정이')
            elif np.argmax(logits) == 1:
              test_eval.append('긍정이')
            elif np.argmax(logits) == 2:
              test_eval.append('무감정이')

        print('>> 입력하신 내용에서 ' + test_eval[0] + ' 느껴집니다.')

In [22]:
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict_sentence(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 집에 가고싶다
>> 입력하신 내용에서 부정이 느껴집니다.


하고싶은 말을 입력해주세요 : 엄마가 피부과 빨리 갔다오래
>> 입력하신 내용에서 부정이 느껴집니다.


하고싶은 말을 입력해주세요 : 엘리베이터에 갇혔었어
>> 입력하신 내용에서 부정이 느껴집니다.


하고싶은 말을 입력해주세요 : 모기한테 물려서 엄청 고생했어
>> 입력하신 내용에서 부정이 느껴집니다.


하고싶은 말을 입력해주세요 : 빨리 쓰고 쉬어야지
>> 입력하신 내용에서 부정이 느껴집니다.


하고싶은 말을 입력해주세요 : 종강은 언제오냐
>> 입력하신 내용에서 부정이 느껴집니다.


하고싶은 말을 입력해주세요 : 흠
>> 입력하신 내용에서 긍정이 느껴집니다.


하고싶은 말을 입력해주세요 : 잘모르겠다
>> 입력하신 내용에서 부정이 느껴집니다.


하고싶은 말을 입력해주세요 : 0


In [25]:
def predict_sentence(predict_sentence):
    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval = []
        for i in out:
            logits = i
            logits = logits.detach().cpu().numpy()
            probabilities = F.softmax(torch.tensor(logits), dim=-1).numpy()

            neg_prob = probabilities[0] * 100
            pos_prob = probabilities[1] * 100
            neu_prob = probabilities[2] * 100

            test_eval.append(f'부정: {neg_prob:.2f}%, 긍정: {pos_prob:.2f}%, 무감정: {neu_prob:.2f}%')

        print('>> 입력하신 내용에서 ' + test_eval[0] + ' 느껴집니다.')

In [26]:
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict_sentence(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 집에 가고싶다


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


>> 입력하신 내용에서 부정: 89.74%, 긍정: 4.21%, 무감정: 6.05% 느껴집니다.


하고싶은 말을 입력해주세요 : 엄마가 피부과 빨리 갔다오래
>> 입력하신 내용에서 부정: 76.77%, 긍정: 10.34%, 무감정: 12.90% 느껴집니다.


하고싶은 말을 입력해주세요 : 엘리베이터에 갇혔었어
>> 입력하신 내용에서 부정: 96.95%, 긍정: 1.29%, 무감정: 1.76% 느껴집니다.


하고싶은 말을 입력해주세요 : 모기한테 물려서 엄청 고생했ㅇ
>> 입력하신 내용에서 부정: 91.16%, 긍정: 6.86%, 무감정: 1.98% 느껴집니다.


하고싶은 말을 입력해주세요 : 빨리 쓰고 쉬어야지
>> 입력하신 내용에서 부정: 42.05%, 긍정: 27.38%, 무감정: 30.57% 느껴집니다.


하고싶은 말을 입력해주세요 : 종강은 언제오나
>> 입력하신 내용에서 부정: 67.51%, 긍정: 17.53%, 무감정: 14.96% 느껴집니다.


하고싶은 말을 입력해주세요 : 흠
>> 입력하신 내용에서 부정: 31.73%, 긍정: 60.03%, 무감정: 8.24% 느껴집니다.


하고싶은 말을 입력해주세요 : 오늘너무재미있었어~!!
>> 입력하신 내용에서 부정: 6.47%, 긍정: 87.73%, 무감정: 5.81% 느껴집니다.


하고싶은 말을 입력해주세요 : 잘 모르겟어..
>> 입력하신 내용에서 부정: 37.54%, 긍정: 16.83%, 무감정: 45.63% 느껴집니다.


하고싶은 말을 입력해주세요 : 0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/models/'
torch.save(model, path+'kobert3.pt')
torch.save(model.state_dict(), path+'kobert3_state_dict.pt')
torch.save({
    'model':model.state_dict(),
    'optimizer':optimizer.state_dict()
}, path+'kobert3_all.tar')

In [ ]:
def predict(model, predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
              test_eval.append('부정이')
            elif np.argmax(logits) == 1:
              test_eval.append('긍정이')
            elif np.argmax(logits) == 2:
              test_eval.append('무감정이')

        print('>> 입력하신 내용에서 ' + test_eval[0] + ' 느껴집니다.')

In [ ]:
predict(tmp, '영화에 나오는 귀신이 너무 무섭네요')

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


>> 입력하신 내용에서 긍정이 느껴집니다.


In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    tmp.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = tmp(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
              test_eval.append('부정이')
            elif np.argmax(logits) == 1:
              test_eval.append('긍정이')
            elif np.argmax(logits) == 2:
              test_eval.append('무감정이')

        print('>> 입력하신 내용에서 ' + test_eval[0] + ' 느껴집니다.')

In [ ]:
predict('영화에 나오는 귀신이 너무 무섭네요')

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


>> 입력하신 내용에서 긍정이 느껴집니다.
